In [4]:
from database.strategy import Strategy
from database.sec import SEC
from database.market import Market
from transformer.date_transformer import DateTransformer
from transformer.model_transformer import ModelTransformer
from transformer.product_transformer import ProductTransformer
from transformer.predictor_transformer import PredictorTransformer
from preprocessor.model_preprocessor import ModelPreprocessor
from preprocessor.predictor_preprocessor import PredictorPreprocessor
from modeler.modeler import Modeler as sp
from utils.date_utils import DateUtils
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timedelta, timezone
from tqdm import tqdm
import math
from IPython.core.debugger import set_trace

In [5]:
## Loading Constants
start = "2008-01-01"
end = datetime.now().strftime("%Y-%m-%d")
# Loading Databases
strat_db = Strategy("portfolio")
market = Market()
sec = SEC("sec")
market.connect()
tickers = market.retrieve_data("sp500").sort_values("Symbol")
market.close()
reload = True
model_range = range(len(tickers["Symbol"]))

In [6]:
sims = []
gap = 365
year_gap = int(gap/365)
quarter_gap = int(gap/90)
year = datetime.now().year
quarter = math.ceil((datetime.now().month - 1) // 3) + 1
training_years = 4
training_days = 100
sec.connect()
strat_db.connect()
quarterly_classification_data = strat_db.retrieve_data("dataset_quarter_classification")
quarterly_regression_data = strat_db.retrieve_data("dataset_quarter_regression")
strat_db.drop_table("fundamental_quarterly_rec")
strat_db.drop_table("fundamental_quarterly_rec_errors")
try:
    ## Setting Up
    date_ranges = DateUtils.create_quarterly_training_range_rec(year,quarter,training_years,gap)
    training_start,training_end,prediction_start,prediction_end = date_ranges
    dates = pd.to_datetime(date_ranges)
    quarters = [x.quarter for x in dates]
    years = [x.year for x in dates]
    print(date_ranges)
    ### switch to all tickers
    for i in tqdm(model_range):
        try:
            ticker = tickers.iloc[i]["Symbol"].replace(".","-")
            if ticker in quarterly_regression_data.columns:
                cik = int(tickers.iloc[i]["CIK"].item())
                price = strat_db.retrieve_price_data("prices",ticker)
                filing = sec.retrieve_filing_data(cik)
                mt = ModelTransformer(ticker,training_start,training_end,gap)
                pt = ModelTransformer(ticker,prediction_start,prediction_end,gap)
                mr = ModelPreprocessor(ticker)
                prot = ProductTransformer(ticker,prediction_start,prediction_end)
                ## regression_model
                rfd = mt.fundamental_merge(price.copy(),filing.copy(),True,classify=False)
                factors = rfd.columns 
                refined = mr.fundamental_preprocess(rfd.copy())
                rfr = sp.regression(refined,ranked=False,tf=True,deep=False)
                cfd = mt.fundamental_merge(price.copy(),filing.copy(),True,classify=True)
                refined = mr.fundamental_preprocess(cfd.copy())
                cfr = sp.classification(refined,tf=True,deep=False)
                fundamental_results = pd.DataFrame([cfr,rfr])
                product_refineds = []
                product_qpds = []
                for i in range(len(fundamental_results)):
                    fundamental_result = fundamental_results.iloc[i]
                    if fundamental_result["model_type"] == "regression":
                        fundamental_data = pt.fundamental_merge_rec(filing.copy(),True,factors)
                    else:
                        fundamental_data = pt.fundamental_merge_rec(filing.copy(),True,factors)
                    product_refined = mr.fundamental_preprocess_rec(fundamental_data.copy())
                    product_refineds.append(product_refined)
                sim = prot.merge_quarterlies_v2(product_refineds,fundamental_results,year,quarter,"fundamental",gap)
                strat_db.store_data("{}_fundamental_quarterly_rec",pd.DataFrame([sim]))
        except Exception as e:
            message = {"status":"quarterly modeling","ticker":ticker,"year":str(year),"quarter":str(quarter),"message":str(e)}
            print(message)
            strat_db.store_data("fundamental_quarterly_rec_errors",pd.DataFrame([message]))
except Exception as e:
    print(year,quarter,str(e))
sec.close()
strat_db.close()

  0%|                                                                                                                                                                                            | 0/506 [00:00<?, ?it/s]

['2017-04-03', '2021-03-31', '2021-04-01', '2021-06-30']
1/1 [==============================] - 0s 2ms/step - loss: 0.5619 - accuracy: 0.7500


  0%|▎                                                                                                                                                                                 | 1/506 [00:07<1:06:37,  7.92s/it]

1/1 [==============================] - 0s 2ms/step - loss: 0.7305 - accuracy: 0.5000


  0%|▋                                                                                                                                                                                 | 2/506 [00:16<1:08:26,  8.15s/it]

...

 99%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▉ | 503/506 [52:12<00:16,  5.41s/it]

1/1 [==============================] - 0s 2ms/step - loss: 0.5983 - accuracy: 0.7500


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▎| 504/506 [52:19<00:11,  5.62s/it]

1/1 [==============================] - 0s 2ms/step - loss: 0.7196 - accuracy: 0.5000


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 506/506 [52:28<00:00,  6.22s/it]


In [7]:
sim

{'year': 2021,
 'quarter': 2,
 'quarterly_fundamental_classification_prediction': 0,
 'quarterly_fundamental_classification_score': 0.75,
 'quarterly_fundamental_regression_prediction': 41.651016,
 'quarterly_fundamental_regression_score': 0.17326906403331976,
 'ticker': 'ZION'}